# Attention Architecture

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import music21

In [3]:
# from fastai.text import *
from enum import Enum
import torch

In [4]:
import numpy as np
import torch.nn as nn

In [5]:
np.set_printoptions(edgeitems=10, threshold=40, linewidth=200)

In [6]:
import sys
sys.path.insert(0, '../../')
from src.fastai_data import *
from src.encode_data import *
from src.serve import *

In [7]:
from src.music_transformer import *

In [8]:
from src.unilm import *

In [9]:
config = v15s_config(vocab)

In [10]:
config['bs'] = 2
config['bptt'] = 1024
config['n_layers'] = 10
config['dec_layers'] = 6

In [11]:
config

{'ctx_len': 150,
 'n_layers': 10,
 'n_heads': 8,
 'd_model': 256,
 'd_head': 32,
 'd_inner': 2048,
 'resid_p': 0.1,
 'attn_p': 0.1,
 'ff_p': 0.1,
 'embed_p': 0.1,
 'output_p': 0.1,
 'bias': False,
 'scale': True,
 'act': <Activation.GeLU: 3>,
 'double_drop': True,
 'tie_weights': True,
 'out_bias': True,
 'init': <function fastai.text.models.transformer.init_transformer(m)>,
 'mem_len': 512,
 'mask': True,
 'pad_idx': 1,
 'bos_idx': 0,
 'sep_idx': 8,
 'transpose_range': (0, 12),
 'note_range': (9, 138),
 'bs': 2,
 'bptt': 1024,
 'vocab_size': 274,
 'dec_layers': 6}

In [12]:
base_path = Path('../../data/midi/v15/')

## BERT Dataloading

In [13]:
ns_dl_tfms = [mask_tfm, next_sentence_tfm]
ns_config = config.copy()
ns_config['bs'] *= 2
ns_data = load_music_data(base_path/'piano_duet', cache_name='tmp/sample', vocab=vocab, 
                          y_offset=0, dl_tfms=ns_dl_tfms, **ns_config)

## S2S Dataloading

In [14]:
s2s_dl_tfms = [mask_s2s_tfm]
s2s_data = MusicDataBunch.load(base_path/'s2s_encode', cache_name='tmp/sample', 
                           preloader_cls=S2SPreloader, dl_tfms=[mask_s2s_tfm], y_offset=1,
                           shuffle_dl=True, **config)


### Load NextWord dataset (Language Model)

In [15]:
nw_dl_tfms = [nw_tfm]
nw_data = load_music_data(base_path/'piano_duet', cache_name='tmp/sample', 
                          vocab=vocab, dl_tfms=nw_dl_tfms, y_offset=1, **config)

## Load

In [16]:
metrics = [mask_acc, ns_acc, s2s_acc, nw_acc]
loss_func = BertLoss(CrossEntropyFlat(ignore_index=vocab.pad_idx), 
                           CrossEntropyFlat(), 
                           CrossEntropyFlat(ignore_index=vocab.pad_idx))

learn = bert_model_learner(s2s_data, config.copy(), 
                           loss_func=loss_func, metrics=metrics)

Sep_idx: 8


In [17]:
learn.callbacks = [BertTrainer(learn, ns_data, s2s_data, nw_data)]
# learn.callbacks = [BertTrainer(learn, nw_data, nw_data, nw_data)]

In [18]:
# learn.loss_func = BertLoss(CrossEntropyFlat(ignore_index=vocab.pad_idx), 
#                            CrossEntropyFlat(), CrossEntropyFlat(ignore_index=vocab.pad_idx))

In [19]:
learn = learn.to_fp16()

In [20]:
learn.validate()

RuntimeError: Expected object of scalar type Long but got scalar type Half for argument #2 'vec2'

In [20]:
# learn.lr_find()
# learn.recorder.plot()

In [21]:
xb,yb = s2s_data.one_batch(); xb[0].shape, yb[0].shape

(torch.Size([2, 1024]), torch.Size([2, 1024]))

In [22]:
learn.fit_one_cycle(3, 1e-4)

epoch,train_loss,valid_loss,mask_acc,ns_acc,s2s_acc,nw_acc,time


KeyboardInterrupt: 